In [11]:
import os
os.chdir('/Users/ruanhq/Desktop/Davis_PhD_Study/UCDAVIS/ECS 271/Project/Data')


In [12]:
#Apply the GAT classifier on the T4S1 data:
#Combine the T4(AA) and S1(BB) together to build a classifier.
import pandas as pd
import os
import numpy as np

AA_T4 = pd.read_csv('T4_scRNA.csv', sep = ' ')
BB_S1 = pd.read_csv('S1_scRNA.csv', sep = ' ')
AA_T4 = np.asarray(AA_T4)
BB_S1 = np.asarray(BB_S1)
n_A = AA_T4.shape[0]
n_B = BB_S1.shape[0]
labels_t4s1= np.zeros((n_A + n_B ,2))
labels_t4s1[:n_A,0] = 1
labels_t4s1[n_A:(n_A + n_B), 1] = 1


In [29]:
os.chdir('/Users/Rhq/Desktop/UCDAVIS/ECS 271/Project/Data/cora')


In [13]:
from sklearn.neighbors import kneighbors_graph
#Construct the Adjacency matrix and the whole genomic data:
AB_T4S1 = np.concatenate([AA_T4, BB_S1])
Adj_t4s1 = kneighbors_graph(AB_T4S1, n_neighbors = 50)


In [ ]:
#Simulate another graph topology:


In [14]:
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from lxml import html, etree
import matplotlib.pyplot as plt
from keras.layers import Layer, Dropout, LeakyReLU
from sklearn.metrics import average_precision_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import LSTM, Dense, Activation, Embedding, Dropout, Input, RepeatVector, TimeDistributed, Bidirectional
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras import activations, constraints, initializers, regularizers
from keras import backend as K
from keras.utils import to_categorical
import numpy as np
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2



Using TensorFlow backend.


In [ ]:
class GraphAttention(Layer):

    def __init__(self,
                 F_,
                 attn_heads=1,
                 attn_heads_reduction='concat',  # {'concat', 'average'}
                 dropout_rate=0.5,
                 activation='relu',
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 attn_kernel_initializer='glorot_uniform',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 attn_kernel_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 attn_kernel_constraint=None,
                 **kwargs):
        if attn_heads_reduction not in {'concat', 'average'}:
            raise ValueError('Possbile reduction methods: concat, average')

        self.F_ = F_  # Number of output features (F' in the paper)
        self.attn_heads = attn_heads  # Number of attention heads (K in the paper)
        self.attn_heads_reduction = attn_heads_reduction  # Eq. 5 and 6 in the paper
        self.dropout_rate = dropout_rate  # Internal dropout rate
        self.activation = activations.get(activation)  # Eq. 4 in the paper
        self.use_bias = use_bias

        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.attn_kernel_initializer = initializers.get(attn_kernel_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.attn_kernel_regularizer = regularizers.get(attn_kernel_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.attn_kernel_constraint = constraints.get(attn_kernel_constraint)
        self.supports_masking = False

        # Populated by build()
        self.kernels = []       # Layer kernels for attention heads
        self.biases = []        # Layer biases for attention heads
        self.attn_kernels = []  # Attention kernels for attention heads

        if attn_heads_reduction == 'concat':
            # Output will have shape (..., K * F')
            self.output_dim = self.F_ * self.attn_heads
        else:
            # Output will have shape (..., F')
            self.output_dim = self.F_

        super(GraphAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) >= 2
        F = input_shape[0][-1]

        # Initialize weights for each attention head
        for head in range(self.attn_heads):
            # Layer kernel
            kernel = self.add_weight(shape=(F, self.F_),
                                     initializer=self.kernel_initializer,
                                     regularizer=self.kernel_regularizer,
                                     constraint=self.kernel_constraint,
                                     name='kernel_{}'.format(head))
            self.kernels.append(kernel)

            # # Layer bias
            if self.use_bias:
                bias = self.add_weight(shape=(self.F_, ),
                                       initializer=self.bias_initializer,
                                       regularizer=self.bias_regularizer,
                                       constraint=self.bias_constraint,
                                       name='bias_{}'.format(head))
                self.biases.append(bias)

            # Attention kernels
            attn_kernel_self = self.add_weight(shape=(self.F_, 1),
                                               initializer=self.attn_kernel_initializer,
                                               regularizer=self.attn_kernel_regularizer,
                                               constraint=self.attn_kernel_constraint,
                                               name='attn_kernel_self_{}'.format(head),)
            attn_kernel_neighs = self.add_weight(shape=(self.F_, 1),
                                                 initializer=self.attn_kernel_initializer,
                                                 regularizer=self.attn_kernel_regularizer,
                                                 constraint=self.attn_kernel_constraint,
                                                 name='attn_kernel_neigh_{}'.format(head))
            self.attn_kernels.append([attn_kernel_self, attn_kernel_neighs])
        self.built = True

    def call(self, inputs):
        X = inputs[0]  # Node features (N x F)
        A = inputs[1]  # Adjacency matrix (N x N)

        outputs = []
        for head in range(self.attn_heads):
            kernel = self.kernels[head]  # W in the paper (F x F')
            attention_kernel = self.attn_kernels[head]  # Attention kernel a in the paper (2F' x 1)

            # Compute inputs to attention network
            features = K.dot(X, kernel)  # (N x F')

            # Compute feature combinations
            # Note: [[a_1], [a_2]]^T [[Wh_i], [Wh_2]] = [a_1]^T [Wh_i] + [a_2]^T [Wh_j]
            attn_for_self = K.dot(features, attention_kernel[0])    # (N x 1), [a_1]^T [Wh_i]
            attn_for_neighs = K.dot(features, attention_kernel[1])  # (N x 1), [a_2]^T [Wh_j]

            # Attention head a(Wh_i, Wh_j) = a^T [[Wh_i], [Wh_j]]
            dense = attn_for_self + K.transpose(attn_for_neighs)  # (N x N) via broadcasting

            # Add nonlinearty
            dense = LeakyReLU(alpha=0.2)(dense)

            # Mask values before activation (Vaswani et al., 2017)
            mask = -10e9 * (1.0 - A)
            dense += mask

            # Apply softmax to get attention coefficients
            dense = K.softmax(dense)  # (N x N)

            # Apply dropout to features and attention coefficients
            dropout_attn = Dropout(self.dropout_rate)(dense)  # (N x N)
            dropout_feat = Dropout(self.dropout_rate)(features)  # (N x F')

            # Linear combination with neighbors' features
            node_features = K.dot(dropout_attn, dropout_feat)  # (N x F')

            if self.use_bias:
                node_features = K.bias_add(node_features, self.biases[head])

            # Add output of attention head to final output
            outputs.append(node_features)

        # Aggregate the heads' output according to the reduction method
        if self.attn_heads_reduction == 'concat':
            output = K.concatenate(outputs)  # (N x KF')
        else:
            output = K.mean(K.stack(outputs), axis=0)  # N x F')

        output = self.activation(output)
        return output, dense

    def compute_output_shape(self, input_shape):
        output_shape = input_shape[0][0], self.output_dim
        return output_shape

In [ ]:
#Construct a GAT classifier model:
#Examine the attention coefficients:
N = AB_T4S1.shape[0]
F = AB_T4S1.shape[1]
F_ = 6
n_attn_heads = 6
dropout_rate = 0.4
l2_reg = 1e-3
X_in = Input(shape=(F,))
A_in = Input(shape=(N,))
n_classes = 2

dropout1 = Dropout(dropout_rate)(X_in)
graph_attention_1 = GraphAttention(F_,
                                   attn_heads=n_attn_heads,
                                   attn_heads_reduction='concat',
                                   dropout_rate=dropout_rate,
                                   activation='elu',
                                   kernel_regularizer=l2(l2_reg),
                                   attn_kernel_regularizer=l2(l2_reg))([dropout1, A_in])
AS1, ds1 = Dense(32, activation = 'relu')(graph_attention_1)
graph_attention_2 = GraphAttention(8,
                                   attn_heads=1,
                                   attn_heads_reduction='average',
                                   dropout_rate=dropout_rate,
                                   activation='relu',
                                   kernel_regularizer=l2(l2_reg),
                                   attn_kernel_regularizer=l2(l2_reg))([AS1, A_in])
learned_labels, dense2 = Dense(n_classes, activation = 'softmax')(graph_attention_2)
# Build model
model = Model(inputs = [X_in, A_in], outputs = learned_labels)
models = Model(inputs = [X_in, A_in], outputs = graph_attention_2)

In [115]:
es_callback = EarlyStopping(monitor='weighted_acc', patience=100)
mc_callback = ModelCheckpoint('logs/best_model.h5',
                              monitor='weighted_acc',
                              save_best_only=True,
                              save_weights_only=True)
learning_rate = 1e-3
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           (None, 1851)         0                                            
__________________________________________________________________________________________________
dropout_491 (Dropout)           (None, 1851)         0           input_35[0][0]                   
__________________________________________________________________________________________________
input_36 (InputLayer)           (None, 4777)         0                                            
__________________________________________________________________________________________________
graph_attention_26 (GraphAttent (None, 36)           66744       dropout_491[0][0]                
                                                                 input_36[0][0]                   
__________

In [116]:
history1 = model.fit([AB_T4S1, Adj_t4s1],
          labels_t4s1,
          epochs = 30, 
          batch_size=N,
          shuffle=False,  # Shuffling data means shuffling the whole graph
          callbacks=[es_callback,  mc_callback])

Epoch 1/30
4777/4777 [==============================] - 53s 11ms/step - loss: 1.8449 - acc: 0.4726
Epoch 2/30


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `weighted_acc` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with weighted_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


4777/4777 [==============================] - 8s 2ms/step - loss: 1.3243 - acc: 0.4875
Epoch 3/30
4777/4777 [==============================] - 8s 2ms/step - loss: 1.0030 - acc: 0.5099
Epoch 4/30
4777/4777 [==============================] - 8s 2ms/step - loss: 0.9572 - acc: 0.5102
Epoch 5/30
4777/4777 [==============================] - 7s 2ms/step - loss: 0.8303 - acc: 0.5876
Epoch 6/30
4777/4777 [==============================] - 8s 2ms/step - loss: 0.7483 - acc: 0.5361
Epoch 7/30
4777/4777 [==============================] - 9s 2ms/step - loss: 0.7115 - acc: 0.6473
Epoch 8/30
4777/4777 [==============================] - 9s 2ms/step - loss: 0.6323 - acc: 0.7934
Epoch 9/30
4777/4777 [==============================] - 9s 2ms/step - loss: 0.5534 - acc: 0.7909
Epoch 10/30
4777/4777 [==============================] - 9s 2ms/step - loss: 0.5061 - acc: 0.8752
Epoch 11/30
4777/4777 [==============================] - 8s 2ms/step - loss: 0.4886 - acc: 0.9066
Epoch 12/30
4777/4777 [================

In [117]:
#Now let's visualize the data
Low_dim_representation2 = models.predict([AB_T4S1, Adj_t4s1], batch_size = N)

In [118]:
#Save the csv:
np.savetxt('GAT_feature_T4S1_50.csv', Low_dim_representation, delimiter = ',')

In [119]:
#Visualize the TSNE:
from sklearn.manifold import TSNE
tsne = TSNE(n_components = 2, perplexity = 40)
GAT_tsne_2 = tsne.fit_transform(Low_dim_representation)


In [120]:
#We get a 8-dimensional manifold:
Low_dim_representation.shape

(4777, 8)

In [ ]:
#Visualize the attention coefficient:


In [121]:
np.savetxt("GAT_feature_T4S1_50.csv", GAT_tsne_2, delimiter = ',')

In [104]:
#For another data:
from keras.utils import plot_model
plot_model(model, to_file = 'CLASF.png')

In [125]:
#Second application for the NKT cell dataset:
os.chdir('/Users/Rhq/Desktop/UCDAVIS/ECS 271/Project/Data')
clus_info = np.asarray(pd.read_csv('clust_info.csv', sep = ' '))

In [128]:
from keras.utils import to_categorical
label2 = to_categorical(clus_info)

In [131]:
label2 = label2[:, 1:]
label2.shape

(203, 4)

In [133]:
np.sum(Adj_t4s1) / 2

119425.0

In [137]:
NK_expr = pd.read_csv('NK_hvg.csv', sep = ' ')

In [163]:
NK_expr = np.asarray(NK_expr)
n_A = 50
n_B = 50
n_C = 50
n_D = 53
labels_nk= np.zeros((n_A + n_B + n_C + n_D ,4))
labels_nk[:n_A,0] = 1
labels_nk[n_A:(n_A + n_B), 1] = 1
labels_nk[(n_A + n_B):(n_A + n_B + n_C), 1] = 1
labels_nk[(n_A + n_B + n_C):(n_A + n_B + n_C + n_D), 1] = 1



In [145]:
#It only have 203 samples so we design a super simple architecture:
Adj_NK = kneighbors_graph(NK_expr, n_neighbors = 20)

In [165]:
#Construct a model:
N = NK_expr.shape[0]
F = NK_expr.shape[1]
F_ = 2
n_attn_heads = 2
dropout_rate = 0.2
l2_reg = 1e-3
X_in = Input(shape=(F,))
A_in = Input(shape=(N,))
n_classes = 4

dropout1 = Dropout(dropout_rate)(X_in)
graph_attention_1 = GraphAttention(F_,
                                   attn_heads=n_attn_heads,
                                   attn_heads_reduction='concat',
                                   dropout_rate=dropout_rate,
                                   activation='elu',
                                   kernel_regularizer=l2(l2_reg),
                                   attn_kernel_regularizer=l2(l2_reg))([dropout1, A_in])
AS1 = Dense(2, activation = 'relu')(graph_attention_1)
graph_attention_2 = GraphAttention(F_,
                                   attn_heads=1,
                                   attn_heads_reduction='average',
                                   dropout_rate=dropout_rate,
                                   activation='relu',
                                   kernel_regularizer=l2(l2_reg),
                                   attn_kernel_regularizer=l2(l2_reg))([AS1, A_in])
learned_labels = Dense(n_classes, activation = 'softmax')(graph_attention_2)
# Build model
model = Model(inputs = [X_in, A_in], outputs = learned_labels)
models = Model(inputs = [X_in, A_in], outputs = graph_attention_2)

In [166]:
#es_callback = EarlyStopping(monitor='weighted_acc', patience=100)
mc_callback = ModelCheckpoint('logs/best_model2.h5',
                              monitor='weighted_acc',
                              save_best_only=True,
                              save_weights_only=True)
learning_rate = 1e-4
optimizer = Adam(lr= 1e-6)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_45 (InputLayer)           (None, 2000)         0                                            
__________________________________________________________________________________________________
dropout_542 (Dropout)           (None, 2000)         0           input_45[0][0]                   
__________________________________________________________________________________________________
input_46 (InputLayer)           (None, 203)          0                                            
__________________________________________________________________________________________________
graph_attention_36 (GraphAttent (None, 4)            8012        dropout_542[0][0]                
                                                                 input_46[0][0]                   
__________

In [167]:
history1 = model.fit([NK_expr, Adj_NK], labels_nk,
          epochs = 30, 
          batch_size=N,
          shuffle=False,  # Shuffling data means shuffling the whole graph
          callbacks=[es_callback,  mc_callback])

Epoch 1/30
203/203 [==============================] - 37s 180ms/step - loss: 3.6283 - acc: 0.7241
Epoch 2/30
203/203 [==============================] - 0s 115us/step - loss: 2.8141 - acc: 0.6355
Epoch 3/30
203/203 [==============================] - 0s 246us/step - loss: 2.7668 - acc: 0.7537
Epoch 4/30
203/203 [==============================] - 0s 353us/step - loss: 3.2662 - acc: 0.7537
Epoch 5/30
203/203 [==============================] - 0s 178us/step - loss: 3.7022 - acc: 0.7537
Epoch 6/30
203/203 [==============================] - 0s 165us/step - loss: 3.5582 - acc: 0.7488
Epoch 7/30
203/203 [==============================] - 0s 266us/step - loss: 2.6422 - acc: 0.7537
Epoch 8/30
203/203 [==============================] - 0s 127us/step - loss: 3.9180 - acc: 0.7537
Epoch 9/30
203/203 [==============================] - 0s 157us/step - loss: 2.7027 - acc: 0.4089
Epoch 10/30
203/203 [==============================] - 0s 196us/step - loss: 3.6283 - acc: 0.7537
Epoch 11/30
203/203 [=======

In [168]:
Low_dim_repre = models.predict([NK_expr, Adj_NK], batch_size = N)

In [170]:
np.savetxt('NK_2dim.csv', Low_dim_repre, delimiter = ',')

In [171]:
plot_model(model, to_file = 'NK_arc.png')

In [ ]:
#Evaluate the purity issue for attention coefficients:

